# Попытка улучшить показатели лучшей реализации с ошибкой в ~12.93% RMSE
Передача данных as is не увенчалась успехом, поэтому тут тестируем следующие варианты:
1. Попытка более умно подойти к вопросу нормализации фичей (не просто mean/median)
2. Пробуем изменить one hot encoding категориальных фич на target encoder / feature embedding
3. Optuna для тюнинга гиперпараметров
4. Ансамбли моделей.

In [1]:
from IPython.lib.deepreload import reload
from xgboost import train
%load_ext autoreload
%autoreload 2

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from catboost import CatBoostRegressor
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, PowerTransformer, RobustScaler, QuantileTransformer, PolynomialFeatures, KBinsDiscretizer

# Подключаем классы для ансамблей стэкинга
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import StackingRegressor
# Для бэгинга
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from utils.data_manager import DataManager
from utils.model_manager import ModelManager

## 1. Загрузка данных

In [2]:
data_path = 'data/home-data-for-ml-course'
train_data = pd.read_csv(data_path + '/train.csv')
test_data = pd.read_csv(data_path + '/test.csv')

## 2. Предобработка данных 

In [3]:
dm = DataManager()

### 2.1. Удаление ненужных столбцов

In [4]:
intuitively_bad_features = [
    'LotShape',  # Общая форма участка
    'LandContour',  # Рельеф участка
    'LotConfig',  # Конфигурация участка
    'LandSlope',  # Уклон участка
    'MiscFeature',
    'MiscVal',
]
bad_columns = dm.get_all_nan_cols(train_data)
bad_columns.append('Id')
bad_columns.extend(intuitively_bad_features)
bad_columns

['Id',
 'LotShape',
 'LandContour',
 'LotConfig',
 'LandSlope',
 'MiscFeature',
 'MiscVal']

In [5]:
train_data = train_data.drop(columns=bad_columns)
train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,Utilities,Neighborhood,Condition1,Condition2,...,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,AllPub,CollgCr,Norm,Norm,...,0,0,0,NaN,NaN,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,AllPub,Veenker,Feedr,Norm,...,0,0,0,NaN,NaN,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,AllPub,CollgCr,Norm,Norm,...,0,0,0,NaN,NaN,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,AllPub,Crawfor,Norm,Norm,...,0,0,0,NaN,NaN,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,AllPub,NoRidge,Norm,Norm,...,0,0,0,NaN,NaN,12,2008,WD,Normal,250000


In [6]:
X, y = dm.split_data_set_to_x_y(train_data, 'SalePrice')
print(X.shape, y.shape)

(1460, 73) (1460,)


### 2.2. Обработка пропущенных значений и нормализация данных через ColumnTransformer и Pipeline

In [7]:
# Получение числовых колонок
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns

# Получение нечисловых колонок (всех остальных)
non_numeric_columns = X.select_dtypes(exclude=['float64', 'int64']).columns

numeric_columns, non_numeric_columns

(Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
        'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
        'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
        'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
        'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
        'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
        'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MoSold',
        'YrSold'],
       dtype='object'),
 Index(['MSZoning', 'Street', 'Alley', 'Utilities', 'Neighborhood',
        'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
        'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
        'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
        'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir',
        'Electrical', 'KitchenQual', 'Functional'

In [8]:
# Создаем preprocessor с разными трансформерами для разных типов данных
preprocessor = ColumnTransformer(
    transformers=[
        # Для числовых - импутация средним
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
        ]), numeric_columns),
        
        # Для категориальных - сначала импутация, затем one-hot кодирование
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),  # 'constant': заполняет константой, которую можно указать через параметр fill_value
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), non_numeric_columns)
    ], 
    sparse_threshold=0,  # Это заставит возвращать dense матрицу вместо sparse
    remainder='drop'  # Все остальные столбцы не будут включены в результат; passthrough - оставить нетронутыми
)

### 2.3. Разбиение данных на обучающую и тестовую выборки

In [9]:
RANDOM_STATE = 42

In [20]:
# Катбуст сам выбирает даныне для валидации, поэтому отсекаем данные только для теста
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

# Проверка размера полученных наборов
print(f'Train size: {len(X_train)}')
print(f'Test size: {len(X_test)}')

Train size: 1022
Test size: 438


### 2.4. Преобразование данных
Т.к. сейчас будут ансамбли, то pipeline-ом пользоваться не сможем, поэтому преобразуем данные вручную

In [19]:
# Фиттим только на обучающих данных
preprocessor.fit(X_train)

# Трансформируем обучающие и тестовые данные
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

### 2.5. Обучение модели

#### 2.5.1. Обучение модели с использованием Ансамблей

In [12]:
ITERATIONS = 1000
LEARNING_RATE = 0.05
DEPTH = 6
LOSS = 'RMSE'
EVAL = 'RMSE'

In [13]:
# Определяем базовые модели
estimators = [
    ('lr', LinearRegression()),
    ('rf', RandomForestRegressor(
        n_estimators=ITERATIONS,
        max_depth=DEPTH,
        random_state=RANDOM_STATE,
        verbose=0
    )),
    ('gbr', GradientBoostingRegressor(
        n_estimators=ITERATIONS,
        learning_rate=LEARNING_RATE,
        max_depth=DEPTH,
        loss='squared_error',
        random_state=RANDOM_STATE,
        verbose=0
    )),
    ('cb', CatBoostRegressor(
        iterations=ITERATIONS,
        learning_rate=LEARNING_RATE,
        depth=DEPTH,
        loss_function=LOSS,  # 'RMSE' или 'MAE', чаще RMSE
        random_seed=RANDOM_STATE,
        verbose=0
    )),
]

In [14]:
# Смотрим, какие модели дают откровенно плохой результат - они не нужны в итоговом ансамбле
for name, model in estimators:
    model.fit(X_train_processed, y_train)
    pred = model.predict(X_test_processed)
    rmse = root_mean_squared_error(y_test, pred)
    print(f'{name}: RMSE={rmse:.2f}')

lr: RMSE=28026.02
rf: RMSE=29283.10
gbr: RMSE=25103.72
cb: RMSE=23271.34


In [14]:
# Финальные только хорошие модели
estimators = [
    ('cb', CatBoostRegressor(
        iterations=ITERATIONS,
        learning_rate=LEARNING_RATE,
        depth=DEPTH,
        loss_function=LOSS,
        random_seed=RANDOM_STATE,
        verbose=0
    )),
    # ('gbr', GradientBoostingRegressor(
    #     n_estimators=ITERATIONS,
    #     learning_rate=LEARNING_RATE * 0.1,  # часто лучше меньший lr
    #     max_depth=DEPTH // 2,  # меньше глубина помогает избежать переобучения
    #     random_state=RANDOM_STATE,
    #     verbose=0
    # )),
]

In [15]:
# Создаем модель стэкинга
stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=RidgeCV()
)

# Обучаем модель
stacking_regressor.fit(X_train_processed, y_train)

# Предсказание и оценка
y_pred_cat = stacking_regressor.predict(X_test_processed)
mse_cat = mean_squared_error(y_test, y_pred_cat)
rmse_cat = root_mean_squared_error(y_test, y_pred_cat)
r2_cat = r2_score(y_test, y_pred_cat)

mean_target = np.mean(y_test)

print("\n=== Ansamble ===")
print(f'RMSE: {rmse_cat:.4f}')
print(f'R² (коэффициент детерминации): {r2_cat:.4f}')
print(f'Относительная среднеквадратичная ошибка (Relative RMSE): {rmse_cat/mean_target*100:.2f}%')


=== Ansamble ===
RMSE: 23019.8614
R² (коэффициент детерминации): 0.9241
Относительная среднеквадратичная ошибка (Relative RMSE): 12.79%


### Удивительно, но факт - вот этот вариант, где мы загнали в стекинг одну модель катбуста - оказался самым лучшим из всех экспериментов. Потому что катбуст сам по себе ансамблевый метод и не нуждается в дополнительных моделях. А все, что тут произошло, почему на пару долей десятых улучшился показатель, потому что стекинг сгладил результаты своей нормализацией через RidgeCV.

In [16]:
# Формируем Submission файл для Kaggle обучив модель на 100% данных
X_submission = test_data.drop(columns=bad_columns)
X_submission_processed = preprocessor.transform(X_submission)
y_submission = stacking_regressor.predict(X_submission_processed)
submission = pd.DataFrame({
    'Id': test_data['Id'],
    'SalePrice': y_submission
})
submission.set_index('Id', inplace=True)
submission.head()

,SalePrice
Id,
1461,124456.383336
1462,159655.895287
1463,185200.203071
1464,191353.832634
1465,194300.650833


In [17]:
submission.to_csv('submission_ansabmled_catboost.csv')

### Ниже всякие эксперименты с беггингом, стекингом и разными моделями и гиперпараметрами

In [17]:
# Определяем базовые модели
estimators = [
    # ('lr', LinearRegression()),
    # ('rf', RandomForestRegressor(
    #     n_estimators=ITERATIONS,
    #     max_depth=DEPTH,
    #     random_state=RANDOM_STATE,
    #     verbose=0
    # )),
    # ('gbr', GradientBoostingRegressor(
    #     n_estimators=ITERATIONS,
    #     learning_rate=LEARNING_RATE,
    #     max_depth=DEPTH,
    #     loss='squared_error',
    #     random_state=RANDOM_STATE,
    #     verbose=0
    # )),
    ('cb', CatBoostRegressor(
        iterations=ITERATIONS,
        learning_rate=LEARNING_RATE,
        depth=DEPTH,
        loss_function=LOSS,  # 'RMSE' или 'MAE', чаще RMSE
        random_seed=RANDOM_STATE,
        verbose=0
    )),
]

# Создаем модель стэкинга
stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=RidgeCV()
)

# Обучаем модель
stacking_regressor.fit(X_train_processed, y_train)

# Предсказание и оценка
y_pred_cat = stacking_regressor.predict(X_test_processed)
mse_cat = mean_squared_error(y_test, y_pred_cat)
rmse_cat = root_mean_squared_error(y_test, y_pred_cat)
r2_cat = r2_score(y_test, y_pred_cat)

mean_target = np.mean(y_test)

print("\n=== Ansamble ===")
print(f'RMSE: {rmse_cat:.4f}')
print(f'R² (коэффициент детерминации): {r2_cat:.4f}')
print(f'Относительная среднеквадратичная ошибка (Relative RMSE): {rmse_cat/mean_target*100:.2f}%')


=== Ansamble ===
RMSE: 23019.8614
R² (коэффициент детерминации): 0.9241
Относительная среднеквадратичная ошибка (Relative RMSE): 12.79%


##### Мы видим, что стэкинг с моделями, которые дают явно худшие результаты, чем catboost, не дает улучшения. Попробуем бэггинг

In [19]:
# Базовый регрессор (CatBoost)
base_regressor = CatBoostRegressor(
    iterations=ITERATIONS,
    learning_rate=LEARNING_RATE,
    depth=DEPTH,
    loss_function=LOSS,
    random_seed=RANDOM_STATE,
    verbose=0
)

# Эксперименты с параметрами
bagging_params = [
    {
        'estimator': base_regressor, 
        'n_estimators': 10, 
        'max_samples': 0.8,
        'max_features': 0.8,
        'random_state': RANDOM_STATE
    },
    {
        'estimator': base_regressor, 
        'n_estimators': 20, 
        'max_samples': 0.8,
        'max_features': 0.8,
        'random_state': RANDOM_STATE
    },
    {
        'estimator': base_regressor, 
        'n_estimators': 30, 
        'max_samples': 0.8,
        'max_features': 0.8,
        'random_state': RANDOM_STATE
    },
    {
        'estimator': base_regressor, 
        'n_estimators': 40, 
        'max_samples': 0.8,
        'max_features': 0.8,
        'random_state': RANDOM_STATE
    },
    {
        'estimator': base_regressor, 
        'n_estimators': 50, 
        'max_samples': 0.8,
        'max_features': 0.8,
        'random_state': RANDOM_STATE
    },
]

mean_target = np.mean(y_test)
for baggin_param in bagging_params:
    # Bagging для регрессии
    bagging_regressor = BaggingRegressor(**baggin_param)
    
    # Обучаем ансамбль
    bagging_regressor.fit(X_train_processed, y_train)
    
    # Предсказание и оценка
    y_pred_cat = bagging_regressor.predict(X_test_processed)
    mse_cat = mean_squared_error(y_test, y_pred_cat)
    rmse_cat = root_mean_squared_error(y_test, y_pred_cat)
    r2_cat = r2_score(y_test, y_pred_cat)

    print("\n=== Ansamble bagging ===")
    print(baggin_param)
    print(f'RMSE: {rmse_cat:.4f}')
    print(f'R² (коэффициент детерминации): {r2_cat:.4f}')
    print(f'Относительная среднеквадратичная ошибка (Relative RMSE): {rmse_cat/mean_target*100:.2f}%')
    print('====================')


=== Ansamble bagging ===
[{'estimator': <catboost.core.CatBoostRegressor object at 0x30bb170b0>, 'n_estimators': 10, 'max_samples': 0.8, 'max_features': 0.8, 'random_state': 42}, {'estimator': <catboost.core.CatBoostRegressor object at 0x30bb170b0>, 'n_estimators': 20, 'max_samples': 0.8, 'max_features': 0.8, 'random_state': 42}, {'estimator': <catboost.core.CatBoostRegressor object at 0x30bb170b0>, 'n_estimators': 30, 'max_samples': 0.8, 'max_features': 0.8, 'random_state': 42}, {'estimator': <catboost.core.CatBoostRegressor object at 0x30bb170b0>, 'n_estimators': 40, 'max_samples': 0.8, 'max_features': 0.8, 'random_state': 42}, {'estimator': <catboost.core.CatBoostRegressor object at 0x30bb170b0>, 'n_estimators': 50, 'max_samples': 0.8, 'max_features': 0.8, 'random_state': 42}]
RMSE: 25457.2847
R² (коэффициент детерминации): 0.9071
Относительная среднеквадратичная ошибка (Relative RMSE): 14.14%

=== Ansamble bagging ===
[{'estimator': <catboost.core.CatBoostRegressor object at 0x30b

##### Результаты бэггинга
Видим, что, если просто увеличивать количество n_estimators, то улучшений нет, потому что все, что мы делаем, создаем новые модели catboost-а, но обученные на меньшем количестве данных и меньшем кол-ве фичей. Потом усредняем все это и получаются результаты хуже, чем просто результаты обучения одного catboost-а. Беггинг должен хорошо сработать на простых моделях, которые не являются сами по себе ансамблевыми (коими является catboost). Попробуем на LinearRegression, чем черт не шутит :)

In [26]:
# Базовый регрессор
base_regressor = LinearRegression()

# Эксперименты с параметрами
bagging_params = [
    {
        'estimator': base_regressor, 
        'n_estimators': 200, 
        'max_samples': 0.5,
        'max_features': 0.8,
        'random_state': RANDOM_STATE
    },
    # {
    #     'estimator': base_regressor, 
    #     'n_estimators': 200, 
    #     'max_samples': 0.5,
    #     'max_features': 0.6,
    #     'random_state': RANDOM_STATE
    # },
    # {
    #     'estimator': base_regressor, 
    #     'n_estimators': 200, 
    #     'max_samples': 0.5,
    #     'max_features': 0.7,
    #     'random_state': RANDOM_STATE
    # },
    # {
    #     'estimator': base_regressor, 
    #     'n_estimators': 200, 
    #     'max_samples': 0.5,
    #     'max_features': 0.8,
    #     'random_state': RANDOM_STATE
    # },
    # {
    #     'estimator': base_regressor, 
    #     'n_estimators': 200, 
    #     'max_samples': 0.5,
    #     'max_features': 0.9,
    #     'random_state': RANDOM_STATE
    # },
]

mean_target = np.mean(y_test)
for baggin_param in bagging_params:
    # Bagging для регрессии
    bagging_regressor = BaggingRegressor(**baggin_param)
    
    # Обучаем ансамбль
    bagging_regressor.fit(X_train_processed, y_train)
    
    # Предсказание и оценка
    y_pred_cat = bagging_regressor.predict(X_test_processed)
    mse_cat = mean_squared_error(y_test, y_pred_cat)
    rmse_cat = root_mean_squared_error(y_test, y_pred_cat)
    r2_cat = r2_score(y_test, y_pred_cat)

    print("\n=== Ansamble bagging ===")
    print(baggin_param)
    print(f'RMSE: {rmse_cat:.4f}')
    print(f'R² (коэффициент детерминации): {r2_cat:.4f}')
    print(f'Относительная среднеквадратичная ошибка (Relative RMSE): {rmse_cat/mean_target*100:.2f}%')
    print('====================')


=== Ansamble bagging ===
{'estimator': LinearRegression(), 'n_estimators': 200, 'max_samples': 0.5, 'max_features': 0.8, 'random_state': 42}
RMSE: 26533.2668
R² (коэффициент детерминации): 0.8991
Относительная среднеквадратичная ошибка (Relative RMSE): 14.74%


##### Снизили хорошо ошибку бэггингом LinearRegression алгоритму. Попробуем еще другие модели

In [29]:
models = [
    ('LinearRegression', LinearRegression()),
    ('DecisionTree', DecisionTreeRegressor(max_depth=10, random_state=RANDOM_STATE)),
    ('Ridge', Ridge(alpha=1.0)),
    ('KNeighborsRegressor', KNeighborsRegressor(n_neighbors=5)),
    ('SVR', SVR(kernel='rbf', C=10.0, epsilon=0.1)),
]

for model in models:
    model_name, base_regressor = model 
    # Эксперименты с параметрами
    bagging_params = [
        {
            'estimator': base_regressor, 
            'n_estimators': 10, 
            'max_samples': 0.8,
            'max_features': 0.7,
            'random_state': RANDOM_STATE
        },
        {
            'estimator': base_regressor, 
            'n_estimators': 20, 
            'max_samples': 0.8,
            'max_features': 0.7,
            'random_state': RANDOM_STATE
        },
        {
            'estimator': base_regressor, 
            'n_estimators': 30, 
            'max_samples': 0.8,
            'max_features': 0.7,
            'random_state': RANDOM_STATE
        },
        {
            'estimator': base_regressor, 
            'n_estimators': 40, 
            'max_samples': 0.8,
            'max_features': 0.7,
            'random_state': RANDOM_STATE
        },
        {
            'estimator': base_regressor, 
            'n_estimators': 50, 
            'max_samples': 0.8,
            'max_features': 0.7,
            'random_state': RANDOM_STATE
        },
    ]
    
    mean_target = np.mean(y_test)
    for baggin_param in bagging_params:
        # Bagging для регрессии
        bagging_regressor = BaggingRegressor(**baggin_param)
        
        # Обучаем ансамбль
        bagging_regressor.fit(X_train_processed, y_train)
        
        # Предсказание и оценка
        y_pred_cat = bagging_regressor.predict(X_test_processed)
        mse_cat = mean_squared_error(y_test, y_pred_cat)
        rmse_cat = root_mean_squared_error(y_test, y_pred_cat)
        r2_cat = r2_score(y_test, y_pred_cat)
    
        print(f"\n=== Ansamble bagging {model_name} ===")
        print(baggin_param)
        print(f'RMSE: {rmse_cat:.4f}')
        print(f'R² (коэффициент детерминации): {r2_cat:.4f}')
        print(f'Относительная среднеквадратичная ошибка (Relative RMSE): {rmse_cat/mean_target*100:.2f}%')
        print('====================')


=== Ansamble bagging LinearRegression ===
{'estimator': LinearRegression(), 'n_estimators': 10, 'max_samples': 0.8, 'max_features': 0.7, 'random_state': 42}
RMSE: 27363.4477
R² (коэффициент детерминации): 0.8927
Относительная среднеквадратичная ошибка (Relative RMSE): 15.20%

=== Ansamble bagging LinearRegression ===
{'estimator': LinearRegression(), 'n_estimators': 20, 'max_samples': 0.8, 'max_features': 0.7, 'random_state': 42}
RMSE: 27656.3438
R² (коэффициент детерминации): 0.8904
Относительная среднеквадратичная ошибка (Relative RMSE): 15.36%

=== Ansamble bagging LinearRegression ===
{'estimator': LinearRegression(), 'n_estimators': 30, 'max_samples': 0.8, 'max_features': 0.7, 'random_state': 42}
RMSE: 27340.6598
R² (коэффициент детерминации): 0.8929
Относительная среднеквадратичная ошибка (Relative RMSE): 15.19%

=== Ansamble bagging LinearRegression ===
{'estimator': LinearRegression(), 'n_estimators': 40, 'max_samples': 0.8, 'max_features': 0.7, 'random_state': 42}
RMSE: 27615

##### Видим, что бэггинг сработал лучше всего на LinearRegression, DecisionTree и Ridge. Попробуем их в стэкинге

In [31]:
estimators = [
    ('LinearRegression', LinearRegression()),
    ('DecisionTree', DecisionTreeRegressor(max_depth=10, random_state=RANDOM_STATE)),
    ('Ridge', Ridge(alpha=1.0)),
    ('KNeighborsRegressor', KNeighborsRegressor(n_neighbors=5)),
    ('SVR', SVR(kernel='rbf', C=10.0, epsilon=0.1)),
]

# Создаем модель стэкинга
stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=RidgeCV()
)

# Обучаем модель
stacking_regressor.fit(X_train_processed, y_train)

# Предсказание и оценка
y_pred_cat = stacking_regressor.predict(X_test_processed)
mse_cat = mean_squared_error(y_test, y_pred_cat)
rmse_cat = root_mean_squared_error(y_test, y_pred_cat)
r2_cat = r2_score(y_test, y_pred_cat)

mean_target = np.mean(y_test)

print("\n=== Ansamble stacking ===")
print(f'RMSE: {rmse_cat:.4f}')
print(f'R² (коэффициент детерминации): {r2_cat:.4f}')
print(f'Относительная среднеквадратичная ошибка (Relative RMSE): {rmse_cat/mean_target*100:.2f}%')


=== Ansamble stacking ===
RMSE: 28389.0368
R² (коэффициент детерминации): 0.8845
Относительная среднеквадратичная ошибка (Relative RMSE): 15.77%


##### Попробуем бэггинг на CatBoost, но с выборкой по данным и фичам близкими к 1

In [32]:
# Базовый регрессор (CatBoost)
base_regressor = CatBoostRegressor(
    iterations=ITERATIONS,
    learning_rate=LEARNING_RATE,
    depth=DEPTH,
    loss_function=LOSS,
    random_seed=RANDOM_STATE,
    verbose=0
)

# Эксперименты с параметрами
bagging_params = [
    {
        'estimator': base_regressor, 
        'n_estimators': 10, 
        'max_samples': 0.9,
        'max_features': 0.9,
        'random_state': RANDOM_STATE
    },
    # {
    #     'estimator': base_regressor, 
    #     'n_estimators': 20, 
    #     'max_samples': 0.8,
    #     'max_features': 0.8,
    #     'random_state': RANDOM_STATE
    # },
    # {
    #     'estimator': base_regressor, 
    #     'n_estimators': 30, 
    #     'max_samples': 0.8,
    #     'max_features': 0.8,
    #     'random_state': RANDOM_STATE
    # },
    # {
    #     'estimator': base_regressor, 
    #     'n_estimators': 40, 
    #     'max_samples': 0.8,
    #     'max_features': 0.8,
    #     'random_state': RANDOM_STATE
    # },
    # {
    #     'estimator': base_regressor, 
    #     'n_estimators': 50, 
    #     'max_samples': 0.8,
    #     'max_features': 0.8,
    #     'random_state': RANDOM_STATE
    # },
]

mean_target = np.mean(y_test)
for baggin_param in bagging_params:
    # Bagging для регрессии
    bagging_regressor = BaggingRegressor(**baggin_param)
    
    # Обучаем ансамбль
    bagging_regressor.fit(X_train_processed, y_train)
    
    # Предсказание и оценка
    y_pred_cat = bagging_regressor.predict(X_test_processed)
    mse_cat = mean_squared_error(y_test, y_pred_cat)
    rmse_cat = root_mean_squared_error(y_test, y_pred_cat)
    r2_cat = r2_score(y_test, y_pred_cat)

    print("\n=== Ansamble bagging ===")
    print(baggin_param)
    print(f'RMSE: {rmse_cat:.4f}')
    print(f'R² (коэффициент детерминации): {r2_cat:.4f}')
    print(f'Относительная среднеквадратичная ошибка (Relative RMSE): {rmse_cat/mean_target*100:.2f}%')
    print('====================')


=== Ansamble bagging ===
{'estimator': <catboost.core.CatBoostRegressor object at 0x30b2473b0>, 'n_estimators': 10, 'max_samples': 0.9, 'max_features': 0.9, 'random_state': 42}
RMSE: 24727.4780
R² (коэффициент детерминации): 0.9124
Относительная среднеквадратичная ошибка (Relative RMSE): 13.74%


##### Попробуем просто обучить N раз catboost без ансамблей и взять усредненные данные

In [36]:
# Сколько моделей будем обучать и усреднять
N_MODELS = 10

# Список для сохранения предсказаний всех моделей
predictions = []

# Обучаем N моделей CatBoost с разными random_seed
models = []
for i in range(N_MODELS):
    seed = RANDOM_STATE + i * 10  # разные random_seed-ы
    
    model = CatBoostRegressor(
        iterations=ITERATIONS,
        learning_rate=LEARNING_RATE,
        depth=DEPTH,
        loss_function=LOSS,
        random_seed=seed,
        verbose=False
    )
    
    # Обучаем модель на всех данных
    model.fit(X_train_processed, y_train)
    models.append(model)
    
    # Предсказания на тесте
    y_pred = model.predict(X_test_processed)
    
    # Добавляем предсказания в список
    predictions.append(y_pred)

# Усредняем предсказания всех моделей
avg_predictions = np.mean(predictions, axis=0)

# Оценка результатов усреднённой модели
rmse = root_mean_squared_error(y_test, avg_predictions)
r2 = r2_score(y_test, avg_predictions)
mean_target = np.mean(y_test)

print("\n=== Усреднённый ансамбль CatBoost ===")
print(f'RMSE: {rmse:.4f}')
print(f'R² (коэффициент детерминации): {r2:.4f}')
print(f'Относительная среднеквадратичная ошибка (Relative RMSE): {rmse/mean_target*100:.2f}%')


=== Усреднённый ансамбль CatBoost ===
RMSE: 23433.0821
R² (коэффициент детерминации): 0.9213
Относительная среднеквадратичная ошибка (Relative RMSE): 13.02%


#### 2.5.2. Стандартное обучение модели без пайплайна с заранее преобразованными данными

In [14]:
cat_model = CatBoostRegressor(
    iterations=ITERATIONS,
    learning_rate=LEARNING_RATE,
    depth=DEPTH,
    loss_function=LOSS,
    random_seed=RANDOM_STATE,
    verbose=False,
    # eval_metric='RMSE'
)

cat_model.fit(
    X_train_processed, y_train,
)

# Предсказание и оценка
y_pred_cat = cat_model.predict(X_test_processed)
mse_cat = mean_squared_error(y_test, y_pred_cat)
rmse_cat = root_mean_squared_error(y_test, y_pred_cat)
r2_cat = r2_score(y_test, y_pred_cat)

mean_target = np.mean(y_test)

print("\n=== CatBoost ===")
print(f'RMSE: {rmse_cat:.4f}')
print(f'R² (коэффициент детерминации): {r2_cat:.4f}')
print(f'Относительная среднеквадратичная ошибка (Relative RMSE): {rmse_cat/mean_target*100:.2f}%')


=== CatBoost ===
RMSE: 23271.3399
R² (коэффициент детерминации): 0.9224
Относительная среднеквадратичная ошибка (Relative RMSE): 12.93%


#### 2.5.2. Стандартное обучение модели c пайлайном

In [23]:
cat_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', CatBoostRegressor(iterations=1000, 
                                learning_rate=0.05, 
                                depth=6, 
                                loss_function='RMSE',
                                verbose=False,
                                random_seed=RANDOM_STATE,
                                ))
])

# Обучение модели
cat_pipeline.fit(X_train, y_train)

# Предсказание и оценка
y_pred_cat = cat_pipeline.predict(X_test)
mse_cat = mean_squared_error(y_test, y_pred_cat)
rmse_cat = root_mean_squared_error(y_test, y_pred_cat)
r2_cat = r2_score(y_test, y_pred_cat)

mean_target = np.mean(y_test)

print("\n=== CatBoost ===")
print(f'RMSE: {rmse_cat:.4f}')
print(f'R² (коэффициент детерминации): {r2_cat:.4f}')
print(f'Относительная среднеквадратичная ошибка (Relative RMSE): {rmse_cat/mean_target*100:.2f}%')

# Используем кросс-валидацию (cross-validation), чтобы результаты были надежными и не зависели от случайного разбиения.
scores = cross_val_score(cat_pipeline, X, y, scoring='neg_root_mean_squared_error', cv=5)
rmse_scores = -scores
print("Средний RMSE на кросс-валидации:", rmse_scores.mean())


=== CatBoost ===
RMSE: 23271.3399
R² (коэффициент детерминации): 0.9224
Относительная среднеквадратичная ошибка (Relative RMSE): 12.93%
Средний RMSE на кросс-валидации: 25867.99820262165
